In [1]:
%cd ..
%load_ext autoreload
%autoreload 2

/home/teo/userdata/SejongMusic


In [3]:
from sejong_music.yeominrak_processing import AlignedScore


class MaskedDataset:
    def __init__(self, aligned_score: AlignedScore):
        self.aligned_score = aligned_score

    def __len__(self):
        return len(self.masked_score)

    def __getitem__(self, idx):
        return self.aligned_score[idx]
      
dataset = MaskedDataset(aligned_score=AlignedScore())

In [23]:
aligned_score = AlignedScore()

In [4]:
len(aligned_score.measure_features[1])

160

In [4]:
import numpy as np
class BeatPosConverter:
  def __init__(self) -> None:
    self.era_group = [(0, 1), (2, 3, 4, 5), (6, 7)]
    self.era2group = {era:group for group, eras in enumerate(self.era_group) for era in eras}
    self.anchor_points =  [
      {0.0:0.0, 2.0: 2.0, 3.0:3.0, 5.0:6.0, 6.0:8.0, 7.0:9.0, 8.0:10.0},
      {0.0:0.0, 1.0:1.0, 2.0:2.0, 3.0:3.0, 6.0:5.0, 7.0:6.0, 8.0:7.0, 9.0:8.0, 10.0:10.0},
    ]
    self.measure_len = [8, 8, 10, 10, 10, 10, 10, 10]
  
  def __call__(self, beat_pos:float, org_era_idx:int, target_era_idx:int):
    org_group, target_group = self.era2group[org_era_idx], self.era2group[target_era_idx]
    if org_group == target_group:
      return beat_pos
    assert abs(org_group - target_group) == 1, f"Only one group difference is allowed. {org_era_idx} -> {target_era_idx}"
    if beat_pos >= self.measure_len[org_era_idx]:
      num_measures = int(beat_pos // self.measure_len[org_era_idx])
      beat_pos -= num_measures * self.measure_len[org_era_idx]
    else:
      num_measures = 0


    lower_group = min(org_group, target_group)
    
    anchor_points = self.anchor_points[lower_group]
    forward = np.array(list(anchor_points.keys()))
    backward = np.array(list(anchor_points.values()))
    if org_group > target_group:
      backward, forward = forward, backward
    conv_beat_pos = np.interp(beat_pos, forward, backward)
    return conv_beat_pos + num_measures * self.measure_len[target_era_idx]
    
                          
    
b_pos_conv = BeatPosConverter()
b_pos_conv(9.0, 5, 7)
    
assert b_pos_conv(0.0, 0, 1) == 0.0
assert b_pos_conv(0.0, 1, 0) == 0.0
assert b_pos_conv(0.0, 0, 2) == 0.0
assert b_pos_conv(0.0, 2, 0) == 0.0
assert b_pos_conv(4.0, 1, 5) == 4.5, b_pos_conv(4.0, 1, 5)
assert abs(b_pos_conv(4.0, 5, 1) - 3.66666) < 0.0001
assert abs(b_pos_conv(5.0, 5, 7) - 4.33333) < 0.0001
assert b_pos_conv(5.0, 7, 5) == 6.0
assert abs(b_pos_conv(4.33333, 7, 5) - 5.0) < 0.0001

In [5]:
b_pos_conv(4.5, 5, 7)

4.0

In [54]:
from tqdm.auto import tqdm
import numpy as np

class EraAlignPairSet:
  pos_conv = BeatPosConverter()
  
  def __init__(self, aligned_score:AlignedScore):
    if isinstance(aligned_score, AlignedScore):
      self.aligned_score = aligned_score
    else:
      self.aligned_score = AlignedScore()
    self.data_pairs = self.collect_dataset()
      
  def collect_dataset(self):
    total_outputs = []
    for front_idx in range(1, len(self.aligned_score.measure_features)-1):
      back_idx = front_idx + 1
      for start_measure in range(len(self.aligned_score.measure_features[front_idx])-4):
        end_measure = start_measure + 4
        pairs = self.get_align_pair_by_part_and_measure(front_idx, back_idx, start_measure, end_measure)
        total_outputs.append(pairs)
    return total_outputs
    

  def get_align_pair_by_part_and_measure(self, front_idx:int, back_idx:int, start_measure:int, end_measure:int):
    assert front_idx < back_idx, f"front_idx should be smaller than back_idx. {front_idx} >= {back_idx}"
    assert start_measure < end_measure, f"start_measure should be smaller than end_measure. {start_measure} >= {end_measure}"
    # if len(self.aligned_score.measure_features[front_idx]) < start_measure: return []
    # if len(self.aligned_score.measure_features[back_idx]) < start_measure: return []
    front_part = self.aligned_score.measure_features[front_idx][start_measure:end_measure]
    front_part = [note for measure in front_part for note in measure]
    back_part = self.aligned_score.measure_features[back_idx][start_measure:end_measure]
    back_part = [note for measure in back_part for note in measure]

    front_part_duration = sum([x[2] for x in front_part])
    back_part_duration = sum([x[2] for x in back_part])
    # print(f"Front duration: {front_part_duration}, Back duration: {back_part_duration}")
    if front_part_duration != self.pos_conv.measure_len[front_idx] * (end_measure - start_measure): return []
    if back_part_duration != self.pos_conv.measure_len[back_idx] * (end_measure - start_measure): return []

    f_accum_position = np.cumsum([0] + [note[2] for note in front_part])
    b_accum_position = np.cumsum([0] + [note[2] for note in back_part])
    pairs = []
    b_note_id = 0
    for f_note_id, f_note in enumerate(front_part):
      f_pos = self.pos_conv(f_accum_position[f_note_id], front_idx, back_idx)
      b_pos = b_accum_position[b_note_id]
      while f_pos > b_pos and b_note_id < len(back_part)-1:
        b_note_id += 1
        b_pos = b_accum_position[b_note_id]
      b_note = back_part[b_note_id]
      is_transferred = f_note[1] == b_note[1]
      pairs.append( (f_note, is_transferred))
    return pairs
  
  def __len__(self):
    return len(self.data_pairs)
  
  @staticmethod
  def convert_duration(dur:float):
    if dur < 0.5: return 'veryshort'
    elif dur < 1.0: return 'short'
    elif dur < 2.0: return 'long'
    else: return 'verylong'
  
  def _convert_note_feature_to_str_list(self, note):
    return [f"era{note[0]}", f"pitch{int(note[1])}", self.convert_duration(note[2]), note[4]]
   
  def __getitem__(self, idx):
    sequence = self.data_pairs[idx]
    notes = [self._convert_note_feature_to_str_list(x[0]) for x in sequence]
    labels = [x[1] for x in sequence]
    
    return notes, labels

dataset = EraAlignPairSet(aligned_score=aligned_score)
# dataset.get_align_pair_by_part_and_measure(4,5, 40, 45)
dataset[0]

([['era1', 'pitch39', 'veryshort', 'strong'],
  ['era1', 'pitch51', 'verylong', 'strong'],
  ['era1', 'pitch51', 'verylong', 'strong'],
  ['era1', 'pitch39', 'veryshort', 'strong'],
  ['era1', 'pitch51', 'verylong', 'strong'],
  ['era1', 'pitch51', 'long', 'strong'],
  ['era1', 'pitch53', 'long', 'weak'],
  ['era1', 'pitch51', 'long', 'weak'],
  ['era1', 'pitch39', 'veryshort', 'strong'],
  ['era1', 'pitch48', 'verylong', 'strong'],
  ['era1', 'pitch46', 'verylong', 'middle'],
  ['era1', 'pitch39', 'veryshort', 'strong'],
  ['era1', 'pitch48', 'long', 'strong'],
  ['era1', 'pitch51', 'long', 'weak'],
  ['era1', 'pitch48', 'long', 'weak'],
  ['era1', 'pitch39', 'veryshort', 'strong'],
  ['era1', 'pitch46', 'verylong', 'strong'],
  ['era1', 'pitch44', 'long', 'weak'],
  ['era1', 'pitch46', 'verylong', 'middle'],
  ['era1', 'pitch39', 'veryshort', 'strong'],
  ['era1', 'pitch48', 'long', 'strong'],
  ['era1', 'pitch46', 'long', 'weak'],
  ['era1', 'pitch48', 'long', 'weak']],
 [False,
  T

True

In [18]:
def align_notes(front_part, back_part, front_idx, back_idx):
  f_accum_position = np.cumsum([0] + [note[2] for note in front_part])
  b_accum_position = np.cumsum([0] + [note[2] for note in back_part])
  f_note_id = 0
  b_note_id = 0
  pairs = []
  for f_note_id, f_note in enumerate(front_part):
    f_pos = b_pos_conv(f_accum_position[f_note_id], front_idx, back_idx)
    b_pos = b_accum_position[b_note_id]
    # print(f" f_pos: {f_pos}, b_pos: {b_pos}")
    while f_pos > b_pos and b_note_id < len(back_part)-1:
      b_note_id += 1
      b_pos = b_accum_position[b_note_id]
    b_note = back_part[b_note_id]
    is_transferred = f_note[1] == b_note[1]
    pairs.append( (f_note, is_transferred))
  return pairs


  
pairs

[([3, 51.0, 1.0, 0.0, 'strong', 1], True),
 ([3, 39.0, 0.0, 1.0, 'weak', 1], True),
 ([3, 51.0, 2.0, 1.0, 'weak', 1], False),
 ([3, 51.0, 3.0, 3.0, 'middle', 1], True),
 ([3, 39.0, 0.0, 6.0, 'strong', 1], False),
 ([3, 51.0, 2.0, 6.0, 'strong', 1], True),
 ([3, 48.0, 1.0, 8.0, 'middle', 1], True),
 ([3, 53.0, 1.0, 9.0, 'weak', 1], True),
 ([3, 39.0, 0.0, 0.0, 'strong', 2], False),
 ([3, 51.0, 2.0, 0.0, 'strong', 2], True),
 ([3, 48.0, 1.0, 2.0, 'weak', 2], True),
 ([3, 51.0, 3.0, 3.0, 'middle', 2], True),
 ([3, 39.0, 0.0, 6.0, 'strong', 2], False),
 ([3, 56.0, 2.0, 6.0, 'strong', 2], True),
 ([3, 53.0, 1.0, 8.0, 'middle', 2], False),
 ([3, 51.0, 1.0, 9.0, 'weak', 2], True)]

In [ ]:

'''
while f_note_id < len(front_part) and b_note_id < len(back_part):
    f_pos = b_pos_conv(f_accum_position[f_note_id], front_idx, back_idx)
    b_pos = b_accum_position[b_note_id]
    # print(f" f_pos: {f_pos}, b_pos: {b_pos}")
    if f_pos > b_pos:
      b_note_id += 1
      continue
    elif f_pos < b_pos:
      f_note_id += 1
      continue
    f_note = front_part[f_note_id]
    b_note = back_part[b_note_id]
    if f_note[1] == b_note[1]:
        print(f_note, b_note)
    f_note_id += 1
    b_note_id += 1
'''

In [17]:

from music21 import converter
from pathlib import Path
from typing import List, Union
import pickle
from sejong_music.yeominrak_processing import Part


class T5Dataset:
  def __init__(self, xml_path) -> None:
    self.score = converter.parse(xml_path)
    # self.parts = [Part(part, i+start_part_idx) for i, part in enumerate(self.score.parts)]
    if Path(xml_path).with_suffix('.pkl').exists():
        self.parts = pickle.load(open(Path(xml_path).with_suffix('.pkl'), 'rb'))
    else:
        self.parts = [Part(part, i) for i, part in enumerate(self.score.parts)]
        pickle.dump(self.parts, open(Path(xml_path).with_suffix('.pkl'), 'wb'))
        
  def __len__(self):
    return len(self.parts)


dataset = T5Dataset('music_score/yeominlak.musicxml')

In [30]:


class AlignedMeasure:
  measure_length = [4, 8, 10, 10, 10, 10, 10, 10]
  
  def __init__(self, measures: List, measure_idx: int) -> None:
    assert len(measures) == 8, f"measures should be 8 but got {len(measures)}"
    self.measures = measures
    self.number = measure_idx
    self.is_valid = self.check_measure_dur(measures)
    
  
  @classmethod
  def check_measure_dur(cls, measures):
    measure_len_by_era = [ round(sum([note.duration for note in era_measure]), 4) for era_measure in measures]
    return measure_len_by_era == cls.measure_length
  
  def __repr__(self) -> str:
    return f"AlignedMeasure({self.number}), valid: {self.is_valid}"


class AlignmentResult:
  def __init__(self, parts: List[Part]):
    self.parts = parts
    self.measures = [AlignedMeasure([part.measures[i] for part in parts], i) for i in range(len(parts[0].measures))]
  

  
parts = dataset.parts

align_result = AlignmentResult(parts)
align_result.measures

[AlignedMeasure(0), valid: True,
 AlignedMeasure(1), valid: True,
 AlignedMeasure(2), valid: True,
 AlignedMeasure(3), valid: True,
 AlignedMeasure(4), valid: True,
 AlignedMeasure(5), valid: True,
 AlignedMeasure(6), valid: True,
 AlignedMeasure(7), valid: True,
 AlignedMeasure(8), valid: True,
 AlignedMeasure(9), valid: True,
 AlignedMeasure(10), valid: True,
 AlignedMeasure(11), valid: True,
 AlignedMeasure(12), valid: True,
 AlignedMeasure(13), valid: True,
 AlignedMeasure(14), valid: False,
 AlignedMeasure(15), valid: False,
 AlignedMeasure(16), valid: False,
 AlignedMeasure(17), valid: False,
 AlignedMeasure(18), valid: False,
 AlignedMeasure(19), valid: False,
 AlignedMeasure(20), valid: False,
 AlignedMeasure(21), valid: False,
 AlignedMeasure(22), valid: False,
 AlignedMeasure(23), valid: True,
 AlignedMeasure(24), valid: True,
 AlignedMeasure(25), valid: True,
 AlignedMeasure(26), valid: True,
 AlignedMeasure(27), valid: True,
 AlignedMeasure(28), valid: True,
 AlignedMeasure

In [16]:
dataset.parts[2].measures

[[Gnote: 51.0, 3.0, 0.0, strong,
  Gnote: 51.0, 3.0, 3.0, middle,
  Gnote: 51.0, 2.0, 6.0, strong,
  Gnote: 48.0, 1.0, 8.0, middle,
  Gnote: 53.0, 1.0, 9.0, weak],
 [Gnote: 51.0, 2.0, 0.0, strong,
  Gnote: 48.0, 1.0, 2.0, weak,
  Gnote: 51.0, 3.0, 3.0, middle,
  Gnote: 56.0, 2.0, 6.0, strong,
  Gnote: 53.0, 1.0, 8.0, middle,
  Gnote: 51.0, 1.0, 9.0, weak],
 [Gnote: 48.0, 2.0, 0.0, strong,
  Gnote: 48.0, 1.0, 2.0, weak,
  Gnote: 46.0, 3.0, 3.0, middle,
  Gnote: 48.0, 1.0, 6.0, strong,
  Gnote: 53.0, 1.0, 7.0, weak,
  Gnote: 51.0, 1.0, 8.0, middle,
  Gnote: 48.0, 1.0, 9.0, weak],
 [Gnote: 46.0, 2.0, 0.0, strong,
  Gnote: 44.0, 1.0, 2.0, weak,
  Gnote: 46.0, 3.0, 3.0, middle,
  Gnote: 51.0, 1.0, 6.0, strong,
  Gnote: 48.0, 1.0, 7.0, weak,
  Gnote: 46.0, 1.0, 8.0, middle,
  Gnote: 44.0, 1.0, 9.0, weak],
 [Gnote: 53.0, 3.0, 0.0, strong,
  Gnote: 53.0, 3.0, 3.0, middle,
  Gnote: 51.0, 2.0, 6.0, strong,
  Gnote: 53.0, 1.0, 8.0, middle,
  Gnote: 56.0, 1.0, 9.0, weak],
 [Gnote: 53.0, 3.0, 0.0, 

In [11]:
front_part, back_part

NameError: name 'front_part' is not defined

In [5]:
dataset[0]

(tensor([[ 6,  1,  1,  1,  1,  1],
         [ 6, 10,  6,  3,  3,  3],
         [ 6, 13,  6,  4,  5,  3],
         [ 6,  6,  4,  5,  5,  3],
         [ 6, 13, 11,  5,  5,  3],
         [ 6, 13,  8, 11,  4,  3],
         [ 6, 10,  6, 15,  5,  3],
         [ 6, 13,  6, 17,  5,  3],
         [ 6,  6,  4, 19,  3,  3],
         [ 6, 13,  8, 19,  3,  3],
         [ 6, 13,  8, 22,  5,  3],
         [ 6, 16,  8, 25,  4,  3],
         [ 6, 14,  8, 29,  5,  3],
         [ 6, 13,  8, 32,  5,  3],
         [ 6, 10,  6,  3,  3,  4],
         [ 6, 13,  6,  4,  5,  4],
         [ 6,  6,  4,  5,  5,  4],
         [ 6, 13,  8,  5,  5,  4],
         [ 6, 13,  8,  7,  5,  4],
         [ 6, 11,  6, 11,  4,  4],
         [ 6, 13,  6, 13,  5,  4],
         [ 6, 10,  8, 15,  5,  4],
         [ 6, 14,  6, 19,  3,  4],
         [ 6, 16,  6, 20,  5,  4],
         [ 6,  6,  4, 22,  5,  4],
         [ 6, 16,  8, 22,  5,  4],
         [ 6, 16,  8, 25,  4,  4],
         [ 6, 14,  8, 29,  5,  4],
         [ 6, 13,  8